In [0]:
print("hello")

In [0]:
spark.sql("SELECT 1").show()

In [0]:
%pip install sentence-transformers langchain langchain-core langchain-community langchain-huggingface faiss-cpu
dbutils.library.restartPython()


In [0]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


In [0]:
# Load FAISS
index_path = "/Volumes/genai_catalog/car_manuals/manual_chunks/faiss_index"
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.load_local(index_path, embedding_model, allow_dangerous_deserialization=True)

# Setup LLM (Option 1: OpenAI)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Option 2 (Offline/Free): You can use HuggingFace LLM like Mistral via transformers if needed

# Build RAG chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

In [0]:
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

# Load free model (CPU or GPU — will auto-select)
hf_pipeline = pipeline(
    task="text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.1",
    max_new_tokens=256,
    do_sample=True,
    temperature=0.5
)

# Wrap for LangChain
llm = HuggingFacePipeline(pipeline=hf_pipeline)


In [0]:
# Run a test query
query = "How can I change the headlight bulb in Audi A4?"
result = rag_chain.invoke(query)

print("Answer:\n", result["result"])
print("\nSources:")
for doc in result["source_documents"]:
    print(f" - [{doc.metadata['language']}] {doc.metadata['source']}")

In [0]:
 %pip install --upgrade numpy

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()